## Using the XBRL Rule and Query Specification (XULE) to create standardized data  
This interactive Python code copies the [free, open-source XULE plugin](https://github.com/xbrlus/xule/) into an [Arelle](https://pypi.org/project/arelle-release/) installation. Click the run button to get started. This step only needs to be completed **once for the session** unless different versions of Arelle or XULE are required. Click the `Show code` link to review/revise the setup script.

In [ ]:
# @title
import os, shutil, sys, site, platform
print('Please wait while Arelle, XULE and some helper packages are installed. \nA XULE version message appears below when the environment is ready.')

# In this example, Arelle and aniso are required to use XULE - get Arelle release details from GitHub (https://github.com/Arelle/arelle/releases).
# Use %pip -q install git+https://git@github.com/Arelle/arelle.git@master to use Arelle's development release
%pip -q install Arelle-release==2.30.11
%pip -q install aniso8601==9.0.1

# 1) locate Arelle's plugin directory (do not modify this location); remove temp and xuledir if they exist
plugindir = site.getsitepackages()[0] + '/arelle/plugin/'
xuledir = plugindir + 'xule/'
xodeldir = plugindir + 'xodel/'
serializerdir = plugindir + 'serializer/'
SimpleXBRLModeldir = plugindir + 'SimpleXBRLModel/'
temp = plugindir + 'temp/'
tempxule = temp + 'plugin/xule/'
tempxodel = temp + 'plugin/xodel/'
tempserializer = temp + 'plugin/serializer/'
tempSimpleXBRLModel = temp + 'plugin/SimpleXBRLModel/'
if os.path.exists(temp):
        if os.path.isdir(temp):
            os.remove(plugindir + 'semanticHash.py')
            shutil.rmtree(xuledir)
            shutil.rmtree(xodeldir)
            shutil.rmtree(serializerdir)
            shutil.rmtree(SimpleXBRLModeldir)
            shutil.rmtree(temp)
else: ''
os.chdir(plugindir)

# 2) copy a XULE release from GitHub (https://github.com/xbrlus/xule/releases) to plugin directory
!git clone --quiet --depth=1 --branch 30041 --single-branch https://github.com/xbrlus/xule.git temp &> /dev/null
shutil.move(temp + 'plugin/semanticHash.py', plugindir)
shutil.move(tempxule, xuledir)
shutil.move(tempxodel, xodeldir)
shutil.move(tempserializer, serializerdir)
shutil.move(tempSimpleXBRLModel, SimpleXBRLModeldir)

# 3) confirm XULE (change -v to -h and re-run to see help contents for Arelle and XULE)
!arelleCmdLine --plugins 'xule' -v

### The Xodel submodule creates XBRL taxonomies and reports

The command below invokes Arelle and processes [an SEC report created with the 2024 US GAAP Taxonomy](https://www.sec.gov/Archives/edgar/data/700764/000107997324001419/vyey_10q-033124.htm) against a compiled XULE .zip file that analyzes concepts in the report, maps facts to a [standardized financial statement taxonomy](https://taxonomies.xbrl.us/standardized-statements/xusss.xsd) and creates a new report that includes a balance sheet, income statement and cash flow, as well as disclosures for the entity, other income and taxes. 

The compiled code uses Xodel - a XULE submodule that builds XBRL taxonomies and creates standardized reports from structured data - to group concepts (including extensions) for improved comparability. Xodel can also adapt data in formats like csv, json, xml, pdf/a and others to use an XBRL taxonomy so the data is standardized and interoperable. For example, a similar process converted 10 years of Federal Energy Regulatory Commission eForms reports submitted by public utilities from Visual FoxPro to inline XBRL.
  
To run the command below, hover the computer cursor in the code cell, then click the run button and follow the prompts - leave each field blank to use defaults for the SEC report above or complete them to override defaults and press enter.  In addition to loading the report based on the URL, other report-specific inputs are created as facts in the standardized output. Here is a short list of [recent SEC reports that can be standardized with the routine below (login to view data)](https://api.xbrl.us/api/v1/report/search?report.source-name=sec&report.document-type=10-K,10-Q&report.base-taxonomy=US+GAAP+2024&fields=report.entry-url,report.filing-date.sort(DESC),report.sic-code,report.accession,report.limit(25)). 

In [ ]:
# Input field variables with defaults
print('Leave fields blank to use default values (in parentheses) or complete to override, then press enter for each.')
SEC_URL = input('SEC report URL (as above): ') or 'https://www.sec.gov/Archives/edgar/data/700764/000107997324001419/vyey_10q-033124.htm'
DATE = input('SEC filing date (2024-10-17): ') or '2024-10-17'
SIC = input('SIC for entity (1311): ') or '1311'
ACCESSION = input('Accession number of SEC report (0001079973-24-001419): ') or '0001079973-24-001419'
SAVE_TO = input('Location for output file (/content/ for Colab):) ') or '/content/'
FILE = input('Name of output file (2024-xuss- plus accession): ') or '2024-xusss-'+ ACCESSION
FILE_TYPE = input('File type (xml default or json): ') or 'xml'

# Xodel using Arelle processor
!arelleCmdLine --plugins 'xodel|transforms/SEC|validate/EFM|inlineXbrlDocumentSet' \
-f $SEC_URL --httpUserAgent='XULE-Arelle (xbrl.us; support@xbrl.us)' \
--xule-rule-set https://raw.githubusercontent.com/xbrlus/xule/jupyter/2024-ugt-norm.zip \
--xule-arg TAXONOMY_LOC=https://taxonomies.xbrl.us/standardized-statements/xusss.xsd \
--xule-arg FILING_DATE=$DATE \
--xule-arg SIC_CODE=$SIC \
--xule-arg FILING_ACCESSION=$ACCESSION \
--xule-arg FILING_URL=$SEC_URL \
--xince-file-type $FILE_TYPE \
--xodel-location $SAVE_TO \
--xule-arg InstanceName=$FILE \
--xule-arg format=$FILE_TYPE \
--xule-run-only createInstance,make_cf_fact_values,make_bs_fact_values,make_FA,make_OA,make_AP,make_AE,make_LTDC,make_OLC,make_UCL,make_LTDNC,make_OLNC,make_LE,make_is_fact_values,make_standard_data,get_dei_data,get_filing_data,get_bus_desc,get_tax_data,get_royalty_data,get_marketing,make_STDC,get_trading_symbol

The Colab environment will take 2-3 minutes to process the first report because the US GAAP Taxonomy needs to be cached by Arelle. Subsequent uses in this session should take less time, depending on the filing's complexity. Colab uses /content/ as session storage and the standardized report will be available for download when the process finishes. Click the folder icon on the left side of the Colab interface to see and download completed reports.

The compiled .zip and a rountine like the one above programmatically create reports added to the [XBRL US Database of Public Filings](https://xbrl.us/filings-database) that can also be [queried with the XBRL API (login to view data)](https://api.xbrl.us/api/v1/report/search?report.source-name=norm&fields=report.entity-name,report.id,report.document-type,report.accepted-timestamp.sort(DESC),report.base-taxonomy,report.html-url,report.limit(25)).